<a href="https://colab.research.google.com/github/data-IA-2022/FAO_Project_Tarik_FengFeng/blob/main/Copie_de_FAO_project_Fengfeng_Tarik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Data Analyse au sein de la FAO (Food and Agriculture Organization)</h1>

<h2>Contexte du projet</h2>

Au sein du service Data Analytics and Prospective de la FAO, vous êtes en charge d'analyser les données produites par les Nations Unies.

Ces analyses conduisent à des rapports de synthèse à destination des décideurs.

Pour êtes également en charge de la partie Prospective, c'est à dire donner des tendances et des évolutions probables quant à la production agricole, à l'alimentation et la nutrition mondiales.

In [11]:
# Copie des fichiers csv du repertoire github et suppréssions des ancien en cas de mise à jour

!rm -r /content/FAO_Project_Tarik_FengFeng
!git clone https://github.com/data-IA-2022/FAO_Project_Tarik_FengFeng.git

Cloning into 'FAO_Project_Tarik_FengFeng'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 125 (delta 14), reused 0 (delta 0), pack-reused 95
Receiving objects: 100% (125/125), 1.18 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [173]:
#importation des librairies python
import os
import pandas as pd
import glob
import itertools 

In [13]:
# Récupération des chemin vers les fichiers csv
csv_path = '/content/FAO_Project_Tarik_FengFeng/fao_2013'
def find_csv_filenames(path_to_dir=os.getcwd(), extension="csv"):
    filenames = []
    for name in glob.glob(f'{path_to_dir}/*.{extension}'):
      filenames.append(name)
    return filenames
all_csv_files= find_csv_filenames(path_to_dir=csv_path)
print(all_csv_files)

['/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_pays-continents.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_cereal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_population.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_vegetal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_animal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_sous_alimentation.csv']


In [33]:
# Création d'un dictionaire avec nom des fichiers csv comme Key et dataframe comme value

col_names= ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole']
df_dict = {}
for f in all_csv_files:
  df_name= f.split("/")[-1].replace(".csv", "").split("_")[-1]
  if "FAOSTAT_2013" in f:
    df_dict[f'{df_name}'] = pd.read_csv(f, encoding='utf-8', quotechar='"', delimiter=',', names= col_names, header=0)
  else:
    df_dict[f'{df_name}'] = pd.read_csv(f, encoding='utf-8', quotechar='"', delimiter=',')

print(df_dict.keys())

dict_keys(['pays-continents', 'cereal', 'population', 'vegetal', 'animal', 'alimentation'])


In [34]:
# Définition des Dataframes

animal= df_dict["animal"]
cereal= df_dict["cereal"]
population= df_dict["population"]
alimentation= df_dict["alimentation"]
vegetal= df_dict["vegetal"]
pays_continents= df_dict["pays-continents"]

In [35]:
# Définir les codes pays comme index des Df
animal.index= animal["Code Pays"]
animal.index.name= "Code_Pays_id"
cereal.index= cereal["Code Pays"]
cereal.index.name= "Code_Pays_id"
population.index= population["Code Pays"]
population.index.name= "Code_Pays_id"
alimentation.index= alimentation["Code Pays"]
alimentation.index.name= "Code_Pays_id"
vegetal.index= vegetal["Code Pays"]
vegetal.index.name= "Code_Pays_id"
pays_continents.index= pays_continents["Code Pays"]
pays_continents.index.name= "Code_Pays_id"

# Nouvelle section

In [36]:
# nb lignes et nb colonnes de nos df
print(animal.shape)
print(cereal.shape)
print(population.shape)
print(alimentation.shape)
print(vegetal.shape)
print(pays_continents.shape)

(37166, 14)
(16057, 14)
(171, 14)
(204, 14)
(104871, 14)
(1941, 7)


In [89]:
# Join population et cereal production et pays continents
df_pop_cerealprod = population.join(cereal, on= "Code Pays",how="left", lsuffix="_pop", rsuffix="_cer" )
df_pop_cerealprodblé = df_pop_cerealprod.query("Produit_cer in ('Blé') and Élément_cer in ('Production')")
pays_continents= pays_continents.query("`Code Groupe de pays` in (5200,5100,5300,5500,5400) ")
#print(pays_continents)
df_pop_cerealprodblé = df_pop_cerealprodblé.join(pays_continents, on= "Code Pays" ,how="left", lsuffix="_popcer", rsuffix="_conti" )
df_pop_cerealprodblé = df_pop_cerealprodblé[["Valeur_pop","Valeur_cer","Pays_pop","Groupe de pays","Code ISO2"]]
print(df_pop_cerealprodblé)

              Valeur_pop  Valeur_cer                            Pays_pop  \
Code_Pays_id                                                               
2                  30552      5169.0                         Afghanistan   
3                   3173       294.0                             Albania   
4                  39208      3299.0                             Algeria   
7                  21472         4.0                              Angola   
9                  41446      9315.0                           Argentina   
...                  ...         ...                                 ...   
235                28934      6842.0                          Uzbekistan   
236                30405         0.0  Venezuela (Bolivarian Republic of)   
249                24407       232.0                               Yemen   
251                14539       274.0                              Zambia   
181                14150        25.0                            Zimbabwe   

           

In [111]:
# Graphiques scatter plot de la production de blé par rapport à la population par pays

# Error divide by zero encountered in log prendre les valeurs supérieur


# Avec plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io
import numpy as np
# set les couleurs
color_map = {
    'Asie': 'red',
    'Europe': 'blue',
    'Afrique': 'green',
    'Amériques': 'yellow',
    'Océanie': 'purple'
}

io.renderers.default='colab'
df = df_pop_cerealprodblé
fig = px.scatter(df, x="Valeur_pop", y="Valeur_cer",title="Production de Blé par Population par Pays", hover_name="Pays_pop" , color="Groupe de pays", color_discrete_map=color_map, log_x=True, log_y=True)
fig.show()

In [123]:
# Creation data frame pour pareto population
# Join population et  pays continents
df_pareto_pop =  population.join(pays_continents, on= "Code Pays",how="left", lsuffix="_pop", rsuffix="_conti" )
# Utilisation des nom de pays comme index
df_pareto_pop.index = list(df_pareto_pop["Pays_conti"])
# Deux colonne pour la dataframe Valeur et groupe de pays
df_pareto_pop  = df_pareto_pop [["Valeur","Groupe de pays","Code ISO2"]]
# On classe la les valeurs de population de façon décroissante
df_pareto_pop = df_pareto_pop.sort_values(by="Valeur",ascending=False)
# Création de la colonne pourcentage cumulé
df_pareto_pop["Pourcentage_cumulé"] = (df_pareto_pop["Valeur"].cumsum()*100)/df_pareto_pop["Valeur"].sum()
print(df_pareto_pop)

                        Valeur Groupe de pays Code ISO2  Pourcentage_cumulé
Chine                  1416667           Asie      F351           20.245834
Inde                   1252140           Asie        IN           38.140384
États-Unis d'Amérique   320051      Amériques        US           42.714288
Indonésie               249866           Asie        ID           46.285167
Brésil                  200362      Amériques        BR           49.148575
...                        ...            ...       ...                 ...
Kiribati                   102        Océanie        KI           99.995984
Antigua-et-Barbuda          90      Amériques        AG           99.997270
Dominique                   72      Amériques        DM           99.998299
Bermudes                    65      Amériques        BM           99.999228
Saint-Kitts-et-Nevis        54      Amériques        KN          100.000000

[171 rows x 4 columns]


In [176]:
# Création d'un dictionaire de couleur en fonction des codes pays pour eviter le groupage de l'argument color dans plot
color_map = {
    'Asie': 'red',
    'Europe': 'blue',
    'Afrique': 'green',
    'Amériques': 'yellow',
    'Océanie': 'purple'
}
color_pays={}
for pays, continent in zip(df_pareto_pop["Code ISO2"], df_pareto_pop["Groupe de pays"]):
  color_pays[pays] = color_map.get(f'{continent}')

print(color_pays)

{'F351': 'red', 'IN': 'red', 'US': 'yellow', 'ID': 'red', 'BR': 'yellow', 'PK': 'red', 'NG': 'green', 'BD': 'red', 'RU': 'blue', 'JP': 'red', 'MX': 'yellow', 'PH': 'red', 'ET': 'green', 'VN': 'red', 'DE': 'blue', 'EG': 'green', 'IR': 'red', 'TR': 'red', 'TH': 'red', 'FR': 'blue', 'GB': 'blue', 'IT': 'blue', 'MM': 'red', 'ZA': 'green', 'KR': 'red', 'TZ': 'green', 'CO': 'yellow', 'ES': 'blue', 'UA': 'blue', 'KE': 'green', 'AR': 'yellow', 'DZ': 'green', 'PL': 'blue', 'SD': 'green', 'UG': 'green', 'CA': 'yellow', 'IQ': 'red', 'MA': 'green', 'AF': 'red', 'VE': 'yellow', 'PE': 'yellow', 'MY': 'red', 'UZ': 'red', 'SA': 'red', 'NP': 'red', 'GH': 'green', 'MZ': 'green', 'KP': 'red', 'YE': 'red', 'AU': 'purple', 'MG': 'green', 'CM': 'green', 'RO': 'blue', 'AO': 'green', 'LK': 'red', 'CI': 'green', 'NE': 'green', 'CL': 'yellow', 'BF': 'green', 'NL': 'blue', 'KZ': 'red', 'MW': 'green', 'EC': 'yellow', 'GT': 'yellow', 'ML': 'green', 'KH': 'red', 'ZM': 'green', 'ZW': 'green', 'SN': 'green', 'TD': 'g

In [170]:
# Graphique de Pareto pour la population cumulée
# set les couleurs
color_map = {
    'Asie': 'red',
    'Europe': 'blue',
    'Afrique': 'green',
    'Amériques': 'yellow',
    'Océanie': 'purple'
}




# Le graphique en bare d'abord
#color="Groupe de pays", color_discrete_map= color_map,
fig = px.bar(df_pareto_pop, x=df_pareto_pop["Code ISO2"], y="Valeur", title='Population (Valeur log) par Pays et Continents',
             color_discrete_map = color_map, color= df_pareto_pop["Groupe de pays"],
             hover_name=df_pareto_pop.index, log_y=True)
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total descending'})
# add the cumulative sum line
#fig.add_scatter(x=df_pareto_pop["Code ISO2"], y=df_pareto_pop["Pourcentage_cumulé"], mode="lines", name="Pourcentage cumulé")
fig.add_trace(go.Scatter(x=df_pareto_pop["Code ISO2"], y=df_pareto_pop["Pourcentage_cumulé"],mode='lines+markers', name="Pourcentage cumulé", line=dict(color="#0000ff")))
# Show the le graphique
fig.show()
